In [3]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as xet
import cv2
from glob import glob
import os

In [6]:
df = pd.read_csv('mabels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


### Labels for YOLO will be center_x , center_y , height and width

In [16]:
# extract yolo labels
def parsing(path):
    parser = xet.parse(path).getroot()
    #extract name of image from xml
    name = parser.find('filename').text
    filename = f'./images/{name}'
    
    #extract width and height
    size = parser.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    
    return filename,width,height

In [18]:
#applying the parsing to the data frame
df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)

In [20]:
# calculating center_x , center_y , bounding box
df['center_x'] = (df['xmax'] + df['xmin']) / (2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin']) / (2*df['height'])

df['bounding_box_width'] = (df['xmax'] - df['xmin']) / df['width']
df['bounding_box_height'] = (df['ymax'] - df['ymin']) / df['height']

In [25]:
# SPlit data into train and test
train = df.iloc[:200]
test = df.iloc[200:]

In [36]:
from shutil import copy

In [42]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bounding_box_width,bounding_box_height
0,./images\N1.xml,1093,1396,645,727,./images/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,./images\N100.xml,134,301,312,350,./images/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,./images\N101.xml,31,139,128,161,./images/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,./images\N102.xml,164,316,216,243,./images/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,./images\N103.xml,813,1067,665,724,./images/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


In [47]:
test_folder = './data_images/test'

values = df[['filename','center_x','center_y','bounding_box_width','bounding_box_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()

In [49]:
test_folder = './data_images/test'

values = df[['filename','center_x','center_y','bounding_box_width','bounding_box_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()